In [1]:
import queue
import threading


class QueueThread(threading.Thread):
    """A simple class to support tasks for queue in thread.

    Attributes:
        func: A function for the target task.
        queue: A queue.Queue object to acquire params for the function.
    """
    def __init__(self, func, queue):
        super().__init__()
        self.func = func
        self.queue = queue
        self.daemon = True

    def run(self):
        """Start function.
        """
        while True:
            # Exit the thread if there is no item in the queue.
            try:
                self.func(*self.queue.get_nowait())
            except queue.Empty:
                return
            self.queue.task_done()

In [2]:
import requests
import time


class CommonRequest():
    """A simple class to support basic requests.

    Attributes:
        retry_c: A list of HTTP status codes for retrying.
        retry_e: A tuple of Exceptions for retrying.
        retry_interval: A integer of the interval (second) between two retry requests.
    """
    retry_c = [
        requests.codes.too_many_requests,
    ]
    retry_e = (
        requests.exceptions.ConnectionError,
        # For Exceptions raised from `retry_c`.
        requests.exceptions.HTTPError,
        requests.exceptions.Timeout,
    )
    retry_interval = 1

    @classmethod
    def common_get(cls, url, params=None, timeout=5, logger=None):
        """A function for common get request.

        Args:
            url: A string of the request url.
            params: A dictionary of the params.
            timeout: A integer of the timeout (seconds).
            logger: A logger object for logging.

        Returns:
            A request.Response object of the target request.
        """
        return cls.retry(
            requests.get,
            logger=logger,
            url=url,
            params=params,
            timeout=timeout
        )

    @classmethod
    def retry(cls, func, logger=None, *args, **kwargs):
        """A helper function to retry requests when encountering errors and exceptions.

        Args:
            func: A function of the target request.
            logger: A logger object for logging errors and exceptions.
            *args: A list of params for the request function.
            **kwargs: A dictionary of params for the request function.

        Returns:
            A request.Response object of the target request.
        """
        while True:
            try:
                response = func(*args, **kwargs)
                if response.status_code in cls.retry_c:
                    response.raise_for_status()
                return response
            except cls.retry_e as e:
                if logger:
                    logger.info(e)
                time.sleep(cls.retry_interval)
                continue

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer


title_similarity_threshold = 0.5

def compare_title(query_title, resp_title, resp_subtitle):
    """Get the titles' text similarity by TFIDF.

    Args:
        query_title: A string of the original title.
        resp_title: A string of the target title.
        resp_subtitle: A string of the target subtitle.

    Returns:
         An integer of the text similarity of the titles.
    """

    # The original title is compared with target title and target title with subtitle.
    corpus = [query_title,
              resp_title,
              f"{resp_title} {resp_subtitle}"]

    # The stop_words is disabled due to the short length of the corpus.
    vect = TfidfVectorizer(min_df=1, stop_words=None)
    tfidf = vect.fit_transform(corpus)
    # The similarity is the maximum of the results.
    similarity = max((tfidf * tfidf.T).A[0, 1:])
    return similarity

In [4]:
import logging
import os
import sys
import time


def get_logger_handler(log_path=None, is_print=False, level=logging.DEBUG):
    """Generate the logger handlers.

    Args:
        log_path: A string of the log file path. None for no file output.
        is_print: A boolean of whether to print the log in console.
        level: A logging Level Object to set the logging level.

    Returns:
        A list of the logger's handlers.
    """
    handlers = list()
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    if is_print:
        handler = logging.StreamHandler(sys.stdout)
        handler.setLevel(level)
        handler.setFormatter(formatter)
        handlers.append(handler)

    if log_path:
        handler = logging.FileHandler(log_path, encoding="utf-8")
        handler.setLevel(level)
        handler.setFormatter(formatter)
        handlers.append(handler)

    return handlers

time_format = time.strftime('%Y-%m-%d %H-%M-%S', time.localtime())
log_path = "./log"
os.makedirs(log_path, exist_ok=True)

book_logger = logging.Logger("Book Scrape", level=logging.INFO)
book_logger.handlers = get_logger_handler(os.path.join(log_path, f"book {time_format}.log"))

net_logger = logging.Logger("Connection", level=logging.INFO)
net_logger.handlers = get_logger_handler(os.path.join(log_path, f"net {time_format}.log"))

In [5]:
import logging
import time


class DebugTimer:
    """A simple class to record runtime of some codes.

    Attributes:
        start_time: A double of the timer's start time.
        end_time: A double of the timer's end time.
        desc: A string of the description of target process.
        print_format: A string of format of print().
        logger: A logger to output the information.
    """
    start_time = 0
    end_time = 0
    desc = str()
    print_format = str()
    logger = logging.Logger("DebugTimer", level=logging.INFO)

    def __init__(self, desc="unknown process", logger=None):
        self.desc = desc
        self.print_format = "The duration of {} is: {} s.\n"
        self.logger.handlers = logger.handlers if logger else get_logger_handler(is_print=True)

    def __enter__(self):
        self.start()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.end()

    def start(self):
        """Start the timer.
        """
        self.start_time = time.perf_counter()

    def end(self):
        """End the timer.
        """
        self.end_time = time.perf_counter()
        self.logger.info(self.print_format.format(self.desc, self.end_time - self.start_time))

In [6]:
from bs4 import BeautifulSoup
import json
import re
import requests


class ScrapeBookParser:
    """A simple class to parse the book information from a Book Scrape page.

    Attributes:
        book_soup: A BeautifulSoup Object of the Book Scrape page.
    """
    rating_str_to_num = {
        "one": 1,
        "two": 2,
        "three": 3,
        "four": 4,
        "five": 5,
    }

    def __init__(self, book_soup):
        self.book_soup = book_soup
        self.book_main_soup = self.book_soup.find("div", "product_main")
        self.book_table_soup = self.book_soup.find("table", "table table-striped")

    def get_title(self):
        return self.book_main_soup.h1.string

    def get_review_num(self):
        return int(self.book_table_soup.find("th", text="Number of reviews").find_next_sibling("td").string)

    def get_rating(self):
        return self.rating_str_to_num[self.book_main_soup.find("p", "star-rating")["class"][1].lower()]

    def get_description(self):
        desc_soup = self.book_soup.find("div", id="product_description")
        return desc_soup.find_next("p").string if desc_soup else None

    def get_price_excl_tax(self):
        return self.book_table_soup.find("th", text="Price (excl. tax)").find_next_sibling("td").string

    def get_price_incl_tax(self):
        return self.book_table_soup.find("th", text="Price (incl. tax)").find_next_sibling("td").string

    def get_product_type(self):
        return self.book_table_soup.find("th", text="Product Type").find_next_sibling("td").string

    def get_category(self):
        return self.book_soup.find("ul", "breadcrumb").find("li", "active").find_previous_sibling("li").a.string

    def get_availability(self):
        return self.book_table_soup.find("th", text="Availability").find_next_sibling("td").string

class GoogleBookParser:
    """A simple class to parse the book information from Google Books API.

    Attributes:
        book_info: A dictionary of the book information from Google Books API.
    """
    def __init__(self, book_info):
        self.book_info = book_info

    def has_title(self):
        return "title" in self.book_info["volumeInfo"]

    def has_self_link(self):
        return "selfLink" in self.book_info

    def get_title(self):
        return self.book_info["volumeInfo"].get("title", "")

    def get_self_link(self):
        return self.book_info["selfLink"]

    def get_subtitle(self):
        return self.book_info["volumeInfo"].get("subtitle", "")

    def get_authors(self):
        return self.book_info["volumeInfo"].get("authors", [])

    def get_authors_str(self, sep=", "):
        return sep.join(self.get_authors())

    def get_average_rating(self):
        return self.book_info["volumeInfo"].get("averageRating", None)

    def get_rating_count(self):
        return self.book_info["volumeInfo"].get("ratingsCount", None)

    def get_page_count(self):
        return self.book_info["volumeInfo"].get("pageCount", None)

    def get_list_price(self):
        list_price = self.book_info.get("saleInfo", dict()).get("listPrice", None)
        return f"{list_price['amount']} {list_price['currencyCode']}" if list_price else None

    def get_retail_price(self):
        retail_price = self.book_info.get("saleInfo", dict()).get("retailPrice", None)
        return f"{retail_price['amount']} {retail_price['currencyCode']}" if retail_price else None

def get_book_item():
    return {
        "Title of Book": ScrapeBookParser.get_title,
        "Number of reviews": ScrapeBookParser.get_review_num,
        "Rating": ScrapeBookParser.get_rating,
        "Description": ScrapeBookParser.get_description,
        "Price (excl. tax)": ScrapeBookParser.get_price_excl_tax,
        "Price (incl. tax)": ScrapeBookParser.get_price_incl_tax,
        "Product Type": ScrapeBookParser.get_product_type,
        "Book category": ScrapeBookParser.get_category,
        "Availability": ScrapeBookParser.get_availability,
    }

def get_book_extra_item():
    return {
        "Google Author": GoogleBookParser.get_authors_str,
        "Google Page Count": GoogleBookParser.get_page_count,
        "Google Rating": GoogleBookParser.get_average_rating,
        "Google Rating Count": GoogleBookParser.get_rating_count,
        "Google Price": lambda p : GoogleBookParser.get_list_price(p) or GoogleBookParser.get_retail_price(p),
    }

def get_book_list_info():
    """Get the main book index information.

    Returns:
        A list which contains total number, first index in one page, last index in one page (i.e. number in one page).
    """
    response = CommonRequest.common_get("http://books.toscrape.com/index.html", logger=net_logger)
    soup = BeautifulSoup(response.content, "html.parser")
    return [int(item.string) for item in soup.find("form", "form-horizontal", method="get").find_all("strong")]

def get_book_url_list(page, book_queue, *args):
    """Get the book urls and put them into target queue for further scrapy.

    Args:
        page: A integer of the target page index.
        book_queue: A queue.Queue Object to put the urls into.
        *args: A list of parameters for further scrapy.
    """
    response = CommonRequest.common_get(f"http://books.toscrape.com/catalogue/page-{page}.html", logger=net_logger)
    soup = BeautifulSoup(response.content, "html.parser")
    urls = [item.a["href"] for item in soup.findAll("article", "product_pod")]

    for url in urls:
        book_queue.put((url, *args, ))

def get_book_info(url, book_info_list, pbar):
    """Get one book detailed information.

    Args:
        url: A string of the sub link of one book.
        book_info_list: A list of all the books' information.
        pbar: A tqdm.tqdm Object for the progress of book scrapy.
    """
    response = CommonRequest.common_get(f"http://books.toscrape.com/catalogue/{url}", logger=net_logger)
    soup = BeautifulSoup(response.content, "html.parser")
    book_parser = ScrapeBookParser(soup)

    book_info = [func(book_parser) for func in get_book_item().values()]

    # Remove the possible Vol. info where # serves as stop mark which misleads the query.
    # For example: I Hate Fairyland, Vol. 1: Madly Ever After (I Hate Fairyland (Compilations) #1-5)
    query_title = re.sub(r"\(.*#.*\)", "", book_parser.get_title())
    book_extra_info = get_book_extra_info(query_title)

    # List append is thread-safe.
    book_info_list.append(book_info + book_extra_info)
    pbar.update(1)

def get_book_extra_info(query_title, check_num=3):
    """Get one book extra information from multi-source.

    Args:
        query_title: A string of the query title to search for.
        check_num: A integer of the number of results to check the similarity.

    Returns:
        A list of the extra information to concat with the basic one.
    """
    google_url = f'https://www.googleapis.com/books/v1/volumes'
    response = CommonRequest.common_get(
        google_url,
        {
            # Note that the query title will be quoted automatically.
            "q": query_title,
            "langRestrict": "en",
        },
        logger=net_logger
    )
    google_books = json.loads(response.content)
    google_books = google_books['items'] if 'items' in google_books.keys() else []

    for book_info in google_books[:check_num]:
        book_parser = GoogleBookParser(book_info)

        # Sometimes, the query results is incomplete, which needs further request.
        if not book_parser.has_title() and book_parser.has_self_link():
            new_response = CommonRequest.common_get(book_parser.get_self_link(), logger=net_logger)
            book_info = json.loads(new_response.content)
            book_parser = GoogleBookParser(book_info)

        # Neglect the book which has no title information.
        if not book_parser.has_title():
            continue

        google_title = book_parser.get_title()
        google_subtitle = book_parser.get_subtitle()

        similarity = compare_title(query_title, google_title, google_subtitle)
        book_logger.info(
            f"\nRequest Title: {query_title}"
            f"\nResponse Title: {google_title}"
            f"\nResponse Subtitle: {google_subtitle}"
            f"\nSimilarity: {similarity}"
            f"\nRequest Url: {response.request.url}"
            f"\n"
        )

        # If the similarity exceeds the threshold, just judge that the two books are the same one.
        # Note that this may lead to some inaccuracy, if threshold is assigned to an unreasonable value, as it is a hyperparameter.
        # Refer to the book log for further fine-tune.
        if similarity > title_similarity_threshold:
            return [func(book_parser) for func in get_book_extra_item().values()]

    return [None for _ in get_book_extra_item().values()]

In [7]:
import pandas
import math
import tqdm


# Headers for CSV file.
headers = list(get_book_item().keys()) + list(get_book_extra_item().keys())
book_info_list = list()
result_filepath = "./Zirui_Zhou+1927924.csv"

list_queue = queue.Queue()
book_queue = queue.Queue()

with DebugTimer("Getting Book List Info"):
    total, _, step = get_book_list_info()
    print(f"The total book number is {total}.")

pbar = tqdm.tqdm(range(total), position=0, leave=True)
pbar.clear()

for idx in range(math.ceil(total/step)):
    list_queue.put((idx+1, book_queue, book_info_list, pbar))

with DebugTimer("Getting Book Url List"):
    for _ in range(10):
        QueueThread(get_book_url_list, list_queue).start()
    list_queue.join()

with DebugTimer("Getting Book Info"):
    for _ in range(20):
        QueueThread(get_book_info, book_queue).start()
    book_queue.join()

with DebugTimer("Writing CSV"):
    pandas.DataFrame(book_info_list, columns=headers).to_csv(result_filepath, sep='\t')

print("Done")

The total book number is 1000.
2022-10-15 21:46:44,326 - DebugTimer - INFO - The duration of Getting Book List Info is: 2.8263077000010526 s.



2022-10-15 21:46:51,472 - DebugTimer - INFO - The duration of Getting Book Url List is: 7.139823500001512 s.



100%|██████████| 1000/1000 [09:45<00:00,  3.50s/it]

2022-10-15 21:56:29,952 - DebugTimer - INFO - The duration of Getting Book Info is: 578.4770907999991 s.

2022-10-15 21:56:30,010 - DebugTimer - INFO - The duration of Writing CSV is: 0.059273400001984555 s.

Done
